### Обработка данных и построение модели

In [101]:
# загрузка библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from  sklearn.ensemble import IsolationForest
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing  import LabelEncoder
from sklearn import linear_model 
from sklearn import tree 
from sklearn import ensemble 
from sklearn import metrics 
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split 
from sklearn.feature_selection import SelectKBest, f_classif

In [102]:
# загрузка подготовленных данных в датафрейм
data = pd.read_csv("./data/data_prepared.csv")
print('Data shape: {}'.format(data.shape))
data.head(5)

Data shape: (308371, 128)


,baths,fireplace,sqft,zipcode,beds,target,Year built,Remodeled year,year_built_log,remodeled_year_log,...,fp_4,fp_5,fp_6,fp_7,fp_8,school_count,school_distance_min,school_distance_avg,school_rating_avg,school_rating_prx
0,3,1,2900.0,28387,4,418000.0,6,0,1.945910,0.000000,...,0,0,0,1,0,8,2.70,5.5,3.2,4.0
1,3,0,1947.0,99216,3,310000.0,6,0,1.945910,0.000000,...,0,0,0,0,0,3,1.01,1.3,2.7,4.0
2,2,1,3000.0,90049,3,2895000.0,64,58,4.174387,4.077537,...,0,0,0,0,0,3,1.19,2.0,6.7,8.0
3,8,1,6457.0,75205,5,2395000.0,19,19,2.995732,2.995732,...,0,0,0,0,0,4,0.10,0.8,6.8,9.0
4,0,0,897.0,19145,2,209000.0,105,0,4.663439,0.000000,...,0,0,0,0,0,0,1.10,2.8,4.2,5.2


In [103]:
# удаляем исходное поле
data = data.drop(labels=['target_log'], axis=1)

Разбиваем выборку

In [104]:
X = data.drop(['target'], axis=1)
y = data['target']
 
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.2)
# 
print('размер тренировочной выборки', X_train.shape[0])
print('размер тестовой выборки', X_test.shape[0])
print('среднее значение целевой переменной на тренировочной выборке', round(y_train.mean(),2))

размер тренировочной выборки 246696
размер тестовой выборки 61675
среднее значение целевой переменной на тренировочной выборке 480175.01


In [105]:
# делим признаки на бинарные и небинарные
a = X_train.describe().loc['max']
binary_columns = list(a[a==1].keys())
nonbinary_columns = list(a[a>1].keys())
nonbinary_columns

['baths',
 'fireplace',
 'sqft',
 'zipcode',
 'beds',
 'Year built',
 'Remodeled year',
 'year_built_log',
 'remodeled_year_log',
 'lot_size',
 'lot_size_log',
 'sqft_log',
 'school_count',
 'school_distance_min',
 'school_distance_avg',
 'school_rating_avg',
 'school_rating_prx']

In [106]:
#Инициализируем стандартизатор MinMaxScaler
scaler = preprocessing.MinMaxScaler()
#Подгоняем параметры стандартизатора (вычисляем среднее и СКО)
scaler.fit(X_train[nonbinary_columns])
#Производим стандартизацию тренировочной выборки
X_train_nb_mm = scaler.transform(X_train[nonbinary_columns])
#Производим стандартизацию тестовой выборки
X_test_nb_mm = scaler.transform(X_test[nonbinary_columns])

#Инициализируем стандартизатор StandardScaler
scaler = preprocessing.StandardScaler()
#Подгоняем параметры стандартизатора (вычисляем среднее и СКО)
scaler.fit(X_train_nb_mm)
#Производим стандартизацию тренировочной выборки
X_train_nb_scaled = scaler.transform(X_train_nb_mm)
#Производим стандартизацию тестовой выборки
X_test_nb_scaled = scaler.transform(X_test_nb_mm)

# соединяем с бинарными признаками
X_train_scaled = np.concatenate((np.array(X_train[binary_columns]), X_train_nb_scaled), axis=1)
X_test_scaled = np.concatenate((np.array(X_test[binary_columns]), X_test_nb_scaled), axis=1)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=binary_columns+nonbinary_columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=binary_columns+nonbinary_columns)

In [107]:
from sklearn.ensemble import GradientBoostingRegressor

Отбираем наиболее значимые для градиентного бустинга признаки для подбора гиперпараметров

In [108]:
gb_columns = ['heating_1', 'heating_4', 'heating_11', 'pool', 'status_1',
       'status_2', '1_story', '2_story', '3_story', 'type_1', 'type_5',
       'type_6', 'type_10', 'type_34', 'type_38', 'type_39', 'baths',
       'fireplace', 'sqft', 'zipcode', 'beds', 'Year built',
       'Remodeled year', 'year_built_log', 'remodeled_year_log',
       'lot_size', 'lot_size_log', 'sqft_log', 'school_count',
       'school_distance_min', 'school_distance_avg', 'school_rating_avg',
       'school_rating_prx']

In [109]:
X_train_scaled = np.array(X_train_scaled[gb_columns])
X_test_scaled = np.array(X_test_scaled[gb_columns])

In [110]:
# Градиентный бустинг
gb = GradientBoostingRegressor(
    learning_rate=0.1, #темп обучения
    n_estimators=120, #число деревьев
    max_depth=12, #максимальная глубина дерева
    min_samples_leaf = 3,
    min_samples_split = 8,
    random_state=42 #генератор случайных чисел
)
gb.fit(X_train_scaled, y_train)
#Предсказываем результат и Выводим значения метрики 
y_train_predict = gb.predict(X_train_scaled)
y_test_predict = gb.predict(X_test_scaled)
#Рассчитываем MAE
print('MAE score: {:.0f} $'.format(metrics.mean_absolute_error(y_test, y_test_predict)))
#Рассчитываем MAPE
print('MAPE score: {:.0f} %'.format(metrics.mean_absolute_percentage_error(y_test, y_test_predict) * 100))
#Рассчитываем коэффициент детерминации для двух выборок
print("Train R^2: {:.3f}".format(metrics.r2_score(y_train, y_train_predict)))
print("Test R^2: {:.3f}".format(metrics.r2_score(y_test, y_test_predict)))

MAE score: 93243 $
MAPE score: 33 %
Train R^2: 0.963
Test R^2: 0.857


# Сериализация модели

In [111]:
regressor = gb
# Производим сериализацию и записываем результат в файл формата pkl
with open('model.pkl', 'wb') as output:
    pickle.dump(regressor, output)

In [112]:
X_test_scaled.shape

(61675, 33)

In [113]:
X_test_scaled[98]

array([ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        1.        ,  1.        ,  0.        ,  0.        ,  1.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  4.76384475, -0.62955159,  5.11783228, -0.68424028,
        2.60658576, -0.54153184, -0.19670525, -0.06462704,  0.57831278,
       -0.03834092,  0.70216147,  2.91016987, -0.47698062,  0.41207558,
        0.01759975,  0.42488951, -0.60889137])

In [114]:
np.array(y_test)[98]

1295000.0